## Importación de librerías

In [ ]:
# Importaciones necesarias

# Selección de dataset
import pandas as pd

# Conjunto de pruebas
from sklearn.model_selection import train_test_split
import numpy as np
from zlib import crc32

# Exploración
import seaborn as sns  # Librería para visualización de datos
import matplotlib.pyplot as plt  # Importar matplotlib para visualización

# Preparación de datos
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

# Transformación de datos
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

# Entrenamiento y evaluación de modelos
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
# Importar clasificadores
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Importaciones de métricas
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score

# Validación cruzada
from sklearn.model_selection import cross_val_score

## Selección de dataset

In [ ]:
# Cargar los datasets
df_only_trivial = pd.read_csv('datasets/OnlyTrivial_dt.csv') # Cargar el dataset de casos triviales
df_only_non_trivial = pd.read_csv('datasets/OnlyNonTrivial_dt.csv') # Cargar el dataset de casos no triviales

# Seleccionar el dataset y crear una copia explícita
dataset = df_only_trivial.copy() # Seleccionar el dataset de casos triviales y crear una copia para manipularlo

## Información de dataset

In [ ]:
# Mostrar las primeras filas del dataset
dataset.head()

In [ ]:
# Mostrar la información del dataset
dataset.info()

In [ ]:
# Descripción estadística del dataset
dataset.describe()

In [ ]:
# Histograma de las características principales
principal_metrics = ['cbo', 'cboModified', 'fanin', 'fanout', 'wmc', 'dit', 'noc', 'rfc', 'lcom', 'lcom*']
# Generar histograma
dataset[principal_metrics].hist(bins=50, figsize=(20, 15))

## Conjunto de pruebas

In [ ]:
# Dividir el dataset en un conjunto de entrenamiento y uno de prueba
train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)

# Mostrar la cantidad de datos en cada conjunto
print("Largo de conjuntos: ")
print("Entrenamiento: ", len(train_set))
print("Prueba: ", len(test_set))

In [ ]:
# Estabilización de los conjuntos de entrenamiento y prueba
def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

# Como el dataset no tiene una columna de identificación, se creará 
# una columna con el índice de cada fila
dataset_with_id = dataset.reset_index()
# Dividir el dataset en entrenamiento y prueba
train_set, test_set = split_train_test_by_id(dataset_with_id, 0.2, "index")

## Exploración

In [ ]:
# Copia de dataset para manipulación
exploration_dataset = train_set.copy()

In [ ]:
# Escalado de las características principales

# Seleccionar las columnas a escalar
columns_to_scale = ['cbo', 'cboModified', 'fanin', 'fanout', 'wmc', 'dit', 'noc', 'rfc', 'lcom', 'lcom*']

# Asegurar que las columnas sean de tipo float64
exploration_dataset[columns_to_scale] = exploration_dataset[columns_to_scale].astype('float64')

# Crear un pipeline para el escalado
pipeline = Pipeline([
    ('scaler', StandardScaler())
])

# Aplicar el escalado a las columnas seleccionadas
exploration_dataset[columns_to_scale] = pipeline.fit_transform(exploration_dataset[columns_to_scale])

# Mapa de calor de correlaciones
plt.figure(figsize=(10, 8))
sns.heatmap(dataset[columns_to_scale].corr(), annot=True, cmap='coolwarm')
plt.show()

## Preparación de datos

In [ ]:
# Limpieza de datos nulos mediante reemplazo por la mediana con SimpleImputer

# Crear un SimpleImputer con la estrategia de reemplazo por la mediana
# Se ocupa la mediana por ser más robusta a valores atípicos
imputer = SimpleImputer(strategy='median') 

# Seleccionar las columnas numéricas
numerical_columns = dataset.select_dtypes(include=[np.number]).columns.tolist()

# Aplicar el imputer a las columnas numéricas
imputer.fit(dataset[numerical_columns])

# Transformar el dataset
dataset[numerical_columns] = imputer.transform(dataset[numerical_columns])

# Verificar si hay valores nulos
dataset.isnull().sum()

In [ ]:
# Paso de variables categóricas a numéricas Parte 1

# Seleccionar las columnas categóricas (Solo se debe ocupar la columna 'type')
categorical_columns = ['type']

# Crear un OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

# Aplicar el OrdinalEncoder a las columnas categóricas
type_encoded = ordinal_encoder.fit_transform(dataset[categorical_columns])

# Mostrar las categorías
ordinal_encoder.categories_

In [ ]:
# Paso de variables categóricas a numéricas Parte 2

# Crear un OneHotEncoder
cat_encoder = OneHotEncoder()

# Aplicar el OneHotEncoder a las columnas categóricas
type_1hot = cat_encoder.fit_transform(dataset[categorical_columns])

# Mostrar las categorías
type_1hot.toarray()

## Transformación de datos

In [ ]:
# Crear un pipeline para la transformación de datos

# Definir pipeline para las columnas numéricas
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

# Seleccionar las columnas numéricas (metricas principales)
numerical_columns = ['cbo', 'cboModified', 'fanin', 'fanout', 'wmc', 'dit', 'noc', 'rfc', 'lcom', 'lcom*']

# Aplicar el pipeline a las columnas numéricas
dataset_transformed = num_pipeline.fit_transform(dataset[numerical_columns])

# Limpiar valores nulos
#dataset_transformed = dataset.dropna(subset=numerical_columns)

# Mostrar el resultado
dataset_transformed

In [ ]:
# Combinar transformaciones

# Seleccionar las columnas categóricas
categorical_columns = ['type']

# Definir las columnas numéricas y categóricas
numerical_columns = ['cbo', 'cboModified', 'fanin', 'fanout', 'wmc', 'dit', 'noc', 'rfc', 'lcom', 'lcom*']

# Crear un ColumnTransformer
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, numerical_columns),
    ('cat', OneHotEncoder(), categorical_columns)
])

# Aplicar el ColumnTransformer al dataset
dataset_prepared = full_pipeline.fit_transform(dataset)

# Mostrar el resultado
dataset_prepared

## Entrenamiento y evaluación de modelos en el conjunto de entrenamiento

### Funciones de entrenamiento

In [ ]:
def train_model(model, x_train, y_train):
    """
    Entrena un modelo con los datos de entrenamiento.
    """
    model.fit(x_train, y_train)
    return model


def evaluate_model(model, x_data, y_data, dataset_name="Entrenamiento"):
    """
    Evalúa un modelo y calcula métricas para el conjunto de datos especificado.
    """
    predictions = model.predict(x_data)
    mse = mean_squared_error(y_data, predictions)
    root_mse = np.sqrt(mse)

    print(f"Resultados en {dataset_name}:")
    print(f"Modelo: {model.__class__.__name__}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {root_mse:.4f}")
    print("-" * 50)

    return predictions, mse, root_mse


def evaluate_model_with_classification_metrics(model, x_data, y_data, dataset_name="Entrenamiento", classification=False):
    """
    Evalúa un modelo y calcula métricas para el conjunto de datos especificado.
    Incluye métricas adicionales si el modelo es de clasificación.
    """
    predictions = model.predict(x_data)

    if classification:
        accuracy = accuracy_score(y_data, predictions)
        precision = precision_score(y_data, predictions)
        recall = recall_score(y_data, predictions)
        f1 = f1_score(y_data, predictions)

        print(f"Resultados en {dataset_name}:")
        print(f"Modelo: {model.__class__.__name__}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print("-" * 50)

        return predictions, accuracy, precision, recall, f1
    else:
        mse = mean_squared_error(y_data, predictions)
        root_mse = np.sqrt(mse)

        print(f"Resultados en {dataset_name}:")
        print(f"Modelo: {model.__class__.__name__}")
        print(f"MSE: {mse:.4f}")
        print(f"RMSE: {root_mse:.4f}")
        print("-" * 50)

        return predictions, mse, root_mse


def train_and_evaluate_model(model, x_train, y_train, x_test=None, y_test=None):
    # Entrenar el modelo
    model.fit(x_train, y_train)

    # Predecir
    predictions = model.predict(x_train)  # En conjunto de entrenamiento
    if x_test is not None:
        test_predictions = model.predict(x_test)

    # Calcular métricas
    mse = mean_squared_error(y_train, predictions)
    root_mse = np.sqrt(mse)

    # Imprimir resultados
    print(f"Modelo: {model.__class__.__name__}")
    print(f"Error cuadrático medio (MSE): {mse:.4f}")
    print(f"Raíz del error cuadrático medio (RMSE): {root_mse:.4f}")
    print("-" * 50)

    return predictions, mse, root_mse

### Linear Regression

In [ ]:
# Regresión Lineal: Entrenamiento

# Instanciar el modelo
lin_reg = LinearRegression()

# Entrenar el modelo
lin_reg = train_model(lin_reg, dataset_prepared, dataset['refactoring'])


In [ ]:
# Regresión Lineal: Evaluación
# Evaluar en conjunto de entrenamiento
lin_train_predictions, lin_train_mse, lin_train_rmse = evaluate_model(lin_reg, dataset_prepared, dataset['refactoring'], dataset_name="Entrenamiento")


### Logistic Regression

In [ ]:
# Regresión Logística: Entrenamiento

# Instanciar el modelo
log_reg = LogisticRegression()

# Entrenar el modelo
log_reg = train_model(log_reg, dataset_prepared, dataset['refactoring'])

In [ ]:
# Regresión Logística: Evaluación
# Evaluar en conjunto de entrenamiento
log_train_predictions, log_train_accuracy, log_train_precision, log_train_recall, log_train_f1 = evaluate_model_with_classification_metrics(log_reg, dataset_prepared, dataset['refactoring'], dataset_name="Entrenamiento", classification=True)

### Decision Tree Regressor

In [ ]:
# Árbol de Decisión: Entrenamiento

# Instanciar el modelo
dt_reg = DecisionTreeRegressor()

# Entrenar el modelo
dt_reg = train_model(dt_reg, dataset_prepared, dataset['refactoring'])

In [ ]:
# Árbol de Decisión: Evaluación
# Evaluar en conjunto de entrenamiento
dt_train_predictions, dt_train_mse, dt_train_rmse = evaluate_model(dt_reg, dataset_prepared, dataset['refactoring'], dataset_name="Entrenamiento")


### Decision Tree Classifier

In [ ]:
# Árbol de Decisión: Entrenamiento

# Instanciar el modelo
dt_clf = DecisionTreeClassifier()

# Entrenar el modelo
dt_clf = train_model(dt_clf, dataset_prepared, dataset['refactoring'])

In [ ]:
# Árbol de Decisión: Evaluación
# Evaluar en conjunto de entrenamiento
dt_train_predictions, dt_train_accuracy, dt_train_precision, dt_train_recall, dt_train_f1 = evaluate_model_with_classification_metrics(dt_clf, dataset_prepared, dataset['refactoring'], dataset_name="Entrenamiento", classification=True)


### Random Forest Regressor

In [ ]:
# Bosque Aleatorio: Entrenamiento

# Instanciar el modelo
rf_reg = RandomForestRegressor()

# Entrenar el modelo
rf_reg = train_model(rf_reg, dataset_prepared, dataset['refactoring'])

In [ ]:
# Bosque Aleatorio: Evaluación
# Evaluar en conjunto de entrenamiento
rf_train_predictions, rf_train_mse, rf_train_rmse = evaluate_model(rf_reg, dataset_prepared, dataset['refactoring'], dataset_name="Entrenamiento")


### Random Forest Classifier

In [ ]:
# Bosque Aleatorio: Entrenamiento

# Instanciar el modelo
rf_clf = RandomForestClassifier()

# Entrenar el modelo
rf_clf = train_model(rf_clf, dataset_prepared, dataset['refactoring'])

In [ ]:
# Bosque Aleatorio: Evaluación
# Evaluar en conjunto de entrenamiento
rf_train_predictions, rf_train_accuracy, rf_train_precision, rf_train_recall, rf_train_f1 = evaluate_model_with_classification_metrics(rf_clf, dataset_prepared, dataset['refactoring'], dataset_name="Entrenamiento", classification=True)

## Validación cruzada

In [ ]:
# Validación cruzada

global_cv = 5

# Función para mostrar los resultados
def display_scores(scores, metric_name):
    print(f"{metric_name}:", scores)

### Regresión Logística

In [ ]:
# Validación cruzada para el modelo de regresión logística
accuracy_scores = cross_val_score(log_reg, dataset_prepared, dataset['refactoring'], scoring='accuracy', cv=global_cv)
precision_scores = cross_val_score(log_reg, dataset_prepared, dataset['refactoring'], scoring='precision', cv=global_cv)
recall_scores = cross_val_score(log_reg, dataset_prepared, dataset['refactoring'], scoring='recall', cv=global_cv)
f1_scores = cross_val_score(log_reg, dataset_prepared, dataset['refactoring'], scoring='f1', cv=global_cv)

In [ ]:

# Mostrar los resultados
print("Regresión Logística:")
print("Accuracy:", accuracy_scores)
print("Precision:", precision_scores)
print("Recall:", recall_scores)
print("F1 Score:", f1_scores)

### Decision Tree Regressor

In [ ]:
# Validación cruzada para el modelo de árbol de decisión
accuracy_scores = cross_val_score(dt_clf, dataset_prepared, dataset['refactoring'], scoring='accuracy', cv=global_cv)
precision_scores = cross_val_score(dt_clf, dataset_prepared, dataset['refactoring'], scoring='precision', cv=global_cv)
recall_scores = cross_val_score(dt_clf, dataset_prepared, dataset['refactoring'], scoring='recall', cv=global_cv)
f1_scores = cross_val_score(dt_clf, dataset_prepared, dataset['refactoring'], scoring='f1', cv=global_cv)

In [ ]:

# Mostrar los resultados
print("Árbol de decisión:")
print("Accuracy:", accuracy_scores)
print("Precision:", precision_scores)
print("Recall:", recall_scores)
print("F1 Score:", f1_scores)

### Random Forest Regressor

In [ ]:
 # Validación cruzada para el modelo de random forest
accuracy_scores = cross_val_score(rf_clf, dataset_prepared, dataset['refactoring'], scoring='accuracy', cv=global_cv)
precision_scores = cross_val_score(rf_clf, dataset_prepared, dataset['refactoring'], scoring='precision', cv=global_cv)
recall_scores = cross_val_score(rf_clf, dataset_prepared, dataset['refactoring'], scoring='recall', cv=global_cv)
f1_scores = cross_val_score(rf_clf, dataset_prepared, dataset['refactoring'], scoring='f1', cv=global_cv)

In [ ]:
# Mostrar los resultados
print("Random Forest:")
print("Accuracy:", accuracy_scores)
print("Precision:", precision_scores)
print("Recall:", recall_scores)
print("F1 Score:", f1_scores)